In [ ]:
#---This program is used to forecast 1 week CDRs based on 55 previous days' data

#---Install these bibs so the program works properly
import pandas as pd
import numpy as np
from numpy import loadtxt
import statsmodels.api as sm
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
#--Loading Cell_ID vector, whose elements tell from which ID the matrix_TS row is the corresponding time-series

Cell_ID = loadtxt('ID_labels.csv', delimiter=',') 
df = pd.read_csv('matrixTS.csv',header=None)
df.head(6)

In [ ]:
#--Defines the Cell ID to be forecasted and plots the corresponding time-series

matrix = df.to_numpy()

ID=60
time_series = matrix[ID][:]
hour = np.linspace(0,24*62,24*62)

plt.figure(figsize=(15,6))
plt.title('Traffic load of Cell ID = %d' % (Cell_ID[60])) 
plt.plot(hour,time_series,'b')

## Defining training and test data sets

In [ ]:
training_size = 55 # in days
ts_training = time_series[0:24*training_size]
hour_training = hour[0:24*training_size]

ts_test = time_series[24*training_size:]
hour_test = hour[24*training_size:]

plt.figure(figsize=(15,6))
plt.title('Training and test data sets of Cell ID = %d' % (Cell_ID[60])) 
plt.plot(hour_training,ts_training,'b')
plt.plot(hour_test,ts_test,'r')

In [ ]:
#---Standardization of the CDR time-series

ts_training_mean = ts_training.mean()
ts_training_std = ts_training.std() 

ts_training_standardized = (ts_training - ts_training_mean) / ts_training_std
ts_test_standardized = (ts_test - ts_training_mean) / ts_training_std

plt.figure(figsize=(15,6))
plt.title('Standardized training and test data sets of Cell ID = %d' % (Cell_ID[60])) 
plt.plot(hour_training,ts_training_standardized,'b')
plt.plot(hour_test,ts_test_standardized,'r')

## Starting SARIMA model

In [ ]:
#---Here we do a grid search on the parameters p,d,q,P,D,Q,S and select the one with smallest AIC criterion

p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 24) for x in list(itertools.product(p, d, q))]

AIC_vector = []
param_vector = []
param_seasonal_vector = []

print("Grid search in progress...")
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(ts_training_standardized,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)
            results = mod.fit()
            AIC_vector.append(results.aic)
            param_vector.append(param)
            param_seasonal_vector.append(param_seasonal)
            
        except: 
            continue

print("Grid search finished.")            
print('Arg min',np.min(AIC_vector),param_vector[np.argmin(AIC_vector)],param_seasonal_vector[np.argmin(AIC_vector)])

In [ ]:
#--Fit model with parameters that provided the best (smallest) AIC

p = param_vector[np.argmin(AIC_vector)][0]
d = param_vector[np.argmin(AIC_vector)][1]
q = param_vector[np.argmin(AIC_vector)][2]

P = param_seasonal_vector[np.argmin(AIC_vector)][0]
D = param_seasonal_vector[np.argmin(AIC_vector)][1]
Q = param_seasonal_vector[np.argmin(AIC_vector)][2]

mod = sm.tsa.statespace.SARIMAX(ts_training_standardized,
                                order=(p, d, q),
                                seasonal_order=(P, D, Q, 24),
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary())

In [ ]:
#---Prediction with trained model

days_predict = 7 #variable that defines how many days to forecast
pred_uc = results.get_forecast(steps=24*days_predict)
pred_ci = pred_uc.conf_int()

plt.figure(figsize=(20,8))
plt.title("Cell ID %d forecast" % (Cell_ID[60])) 
plt.plot(hour_test,ts_test_standardized,'b')
plt.plot(hour_test,pred_uc.predicted_mean,'k--',label="Forecast")
plt.legend(loc='upper left')
plt.xlabel("Hour")
plt.ylabel("CDR")

In [ ]:
#--Ploting the actual result (not considering mean and std)

test_predictions = pred_uc.predicted_mean*ts_training_std+ts_training_mean

plt.figure(figsize=(20,8))
plt.title("Cell ID %d forecast" % (Cell_ID[60])) 

plt.plot(hour_test,test_predictions,'k--',label="Forecast")
plt.plot(hour[24*55:],time_series[24*55:],'b')

plt.legend(loc='upper left')
plt.xlabel("Hour")
plt.ylabel("CDR")

In [ ]:
MAE = mean_absolute_error(time_series[24*55:], test_predictions)
MSE = mean_squared_error(time_series[24*55:], test_predictions)
print('MAE:', "%.3f" % MAE)
print('MSE:', "%.3f" % MSE)